# Experimental Physics - JHOF

### Winter 2022

This is the template for codes that I may use to write lab analyses in the Winter and Spring quarters 2022 of my Experimental Physics classes. I am learning how to use GitHub right now. This code is a shortcut for me to:

* Create plots
* Use fit functions

Instead of having to check the course wiki page every single time. And by the way, $\frac{3}{12} + \frac{3}{4}$ is an indefinite sum.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize
%matplotlib inline

data_filename = 'Example2_Data.tsv'

In [ ]:
x, y, dy = np.loadtxt(data_filename, unpack=True, skiprows=1, usecols=[0,1,2])

fig, ax = plt.subplots(figsize = (10,8))
ax.errorbar(x,y,dy,fmt='k.',capsize = 3)

Change fit function parameters here:

In [ ]:
def linear(p, xvar):
    return p[0] + p[1]*xvar

Change cost functions here:

In [ ]:
def residual(p,func, xvar, yvar, err):
    return (func(p, xvar) - yvar)/err

chisq = sum(residual(guess,linear,x,y,dy)**2)

The code to actually find the best parameters for the defined function:

In [ ]:
def data_fit(p0,func,xvar, yvar, err,tmi=0):
    try:
        fit = optimize.least_squares(residual, p0, args=(func,xvar, yvar, err),verbose=tmi)
    except Exception as error:
        print("Something has gone wrong:",error)
        return p0,np.zeros_like(p0),-1,-1
    pf = fit['x']

    print()

    try:
        cov = np.linalg.inv(fit['jac'].T.dot(fit['jac']))          
        # This computes a covariance matrix by finding the inverse of the Jacobian times its transpose
        # We need this to find the uncertainty in our fit parameters
    except:
        # If the fit failed, print the reason
        print('Fit did not converge')
        print('Result is likely a local minimum')
        print('Try changing initial values')
        print('Status code:', fit['status'])
        print(fit['message'])
        return pf,np.zeros_like(pf),-1,-1
            #You'll be able to plot with this, but it will not be a good fit.

    chisq = sum(residual(pf,func,xvar, yvar, err) **2)
    dof = len(xvar) - len(pf)
    red_chisq = chisq/dof
    pferr = np.sqrt(np.diagonal(cov)) # finds the uncertainty in fit parameters by squaring diagonal elements of the covariance matrix
    print('Converged with chi-squared {:.2f}'.format(chisq))
    print('Number of degrees of freedom, dof = {:.2f}'.format(dof))
    print('Reduced chi-squared {:.2f}'.format(red_chisq))
    print()
    Columns = ["Parameter #","Initial guess values:", "Best fit values:", "Uncertainties in the best fit values:"]
    print('{:<11}'.format(Columns[0]),'|','{:<24}'.format(Columns[1]),"|",'{:<24}'.format(Columns[2]),"|",'{:<24}'.format(Columns[3]))
    for num in range(len(pf)):
        print('{:<11}'.format(num),'|','{:<24.3e}'.format(p0[num]),'|','{:<24.3e}'.format(pf[num]),'|','{:<24.3e}'.format(pferr[num]))
    return pf, pferr, chisq,dof

pf, pferr, chisq, dof = data_fit(guess,linear, x, y, dy)

Many plotting things:

In [ ]:
X = np.linspace(x.min(), x.max(), 500)
ax.plot(X, linear(pf, X), 'r-', label = 'Linear Fit: $f(x)$')



ax.set_title('Some Sample Data with Error Bars')
ax.set_xlabel('x')
ax.set_ylabel('y')

# Here is the text we want to include...
textfit = '$f(x) = A + Bx$ \n' 
textfit += '$A = {:.2f} \pm {:.2f}$ \n'.format(pf[0],pferr[0]) 
textfit +='$B = {:.2f} \pm {:.2f}$ \n'.format(pf[1],pferr[1]) 
textfit += '$\chi^2= {:.1f}$ \n'.format(chisq) 
textfit += '$N = {}$ (dof) \n'.format(dof) 
textfit += '$\chi^2/N = {:.2f}$'.format(chisq/dof) 

#... and below is where we actually place it on the plot
ax.text(0.05, 0.95, textfit, transform=ax.transAxes , fontsize=12,verticalalignment='top')

ax.set_xlim([x.min()-0.5, x.max()+0.5])
  # x.min() is equal to the smallest x value in the entire array, x.max() is equal to the largest
  # Together this ensures that the axes always scale to be just slightly wider than the data.
ax.legend(loc='lower right')
plt.savefig('Example2_Figure1.pdf')
plt.show()